In [1]:
# imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import numpy as np
from scipy.linalg import sqrtm
from torchvision.models import inception_v3
import torchvision.transforms.functional as TF
from PIL import Image
import pandas as pd
import os
import multiprocessing

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import numpy as np
from scipy.linalg import sqrtm
from torchvision.models import inception_v3
from PIL import Image
import pandas as pd
import os
import torch.nn.functional as F
import multiprocessing
import joblib
from joblib import load
import torchvision.models as models

In [2]:
class Generator(nn.Module):
    def __init__(self, input_nc, output_nc):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            # Encoder layers
            nn.Conv2d(input_nc, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(),
            # Residual blocks
            ResidualBlock(128),
            ResidualBlock(128),
            ResidualBlock(128),
            ResidualBlock(128),
            ResidualBlock(128),
            ResidualBlock(128),
            # Decoder layers
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, output_nc, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)
    
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.InstanceNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.InstanceNorm2d(channels)
        )

    def forward(self, x):
        return x + self.conv_block(x)

In [17]:
device = torch.device('mps')

# path to save checkpoint CHANGE EPOCH NOT WHOLE PATH
epoch = 100
checkpoint_path = f'/Users/kieran/Documents/mlpProject/0319_horses2zebras_1e4_batch8/epoch_100.pth'
checkpoint = torch.load(checkpoint_path, map_location=device)

input_nc = 3  # Number of input channels
output_nc = 3  # Number of output channels
# Initialize networks
G_AB = Generator(input_nc, output_nc).to(device)
G_BA = Generator(output_nc, input_nc).to(device)

# Load the state dictionaries
G_AB.load_state_dict(checkpoint['G_AB_state_dict'])
G_BA.load_state_dict(checkpoint['G_BA_state_dict'])

# Ensure the models are in evaluation mode
G_AB.eval()
G_BA.eval()

# Prepare your dataset
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

dataset_A = ImageFolder(root='/Users/kieran/Documents/mlpProject/zebraHorse/trainA', transform=transform)
dataset_B = ImageFolder(root='/Users/kieran/Documents/mlpProject/zebraHorse/trainB', transform=transform)

dataloader_A = DataLoader(dataset_A, batch_size=1, shuffle=False)
dataloader_B = DataLoader(dataset_B, batch_size=1, shuffle=False)

# Generate and save/display images
output_dir = f'/Users/kieran/Documents/mlpProject/0319_horses2zebras_1e4_batch8/generated_images_train_epoch_{epoch}'
os.makedirs(output_dir, exist_ok=True)

with torch.no_grad():
    for i, (real_A, _) in enumerate(dataloader_A):
        real_A = real_A.to(device)
        fake_B = G_AB(real_A)
        save_image(fake_B, os.path.join(output_dir, f'fake_B_{i}.png'))
        if i == 1000:  # Save/display 10 images for demonstration
            break

    for i, (real_B, _) in enumerate(dataloader_B):
        real_B = real_B.to(device)
        fake_A = G_BA(real_B)
        save_image(fake_A, os.path.join(output_dir, f'fake_A_{i}.png'))
        if i == 1000:
            break

print("Generated images have been saved to", output_dir)


Generated images have been saved to /Users/kieran/Documents/mlpProject/0319_horses2zebras_1e4_batch8/generated_images_train_epoch_100


In [16]:
device = torch.device('mps')

# path to save checkpoint CHANGE EPOCH NOT WHOLE PATH
epoch = 100
checkpoint_path = f'/Users/kieran/Documents/mlpProject/0313_50features_128/epoch_100.pth'
checkpoint = torch.load(checkpoint_path, map_location=device)

input_nc = 3  # Number of input channels
output_nc = 3  # Number of output channels
# Initialize networks
G_AB = Generator(input_nc, output_nc).to(device)
G_BA = Generator(output_nc, input_nc).to(device)

# Load the state dictionaries
G_AB.load_state_dict(checkpoint['G_AB_state_dict'])
G_BA.load_state_dict(checkpoint['G_BA_state_dict'])

# Ensure the models are in evaluation mode
G_AB.eval()
G_BA.eval()

# Prepare your dataset
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

dataset_A = ImageFolder(root='/Users/kieran/Documents/mlpProject/zebraHorse/trainA', transform=transform)
dataset_B = ImageFolder(root='/Users/kieran/Documents/mlpProject/zebraHorse/trainB', transform=transform)

dataloader_A = DataLoader(dataset_A, batch_size=1, shuffle=False)
dataloader_B = DataLoader(dataset_B, batch_size=1, shuffle=False)

# Generate and save/display images
output_dir = f'/Users/kieran/Documents/mlpProject/0313_50features_128/generated_images_train_epoch_{epoch}'
os.makedirs(output_dir, exist_ok=True)

with torch.no_grad():
    for i, (real_A, _) in enumerate(dataloader_A):
        real_A = real_A.to(device)
        fake_B = G_AB(real_A)
        save_image(fake_B, os.path.join(output_dir, f'fake_B_{i}.png'))
        if i == 1000:  # Save/display 10 images for demonstration
            break

    for i, (real_B, _) in enumerate(dataloader_B):
        real_B = real_B.to(device)
        fake_A = G_BA(real_B)
        save_image(fake_A, os.path.join(output_dir, f'fake_A_{i}.png'))
        if i == 1000:
            break

print("Generated images have been saved to", output_dir)


Generated images have been saved to /Users/kieran/Documents/mlpProject/0313_50features_128/generated_images_train_epoch_100
